In [ ]:
!pip install selenium webdriver-manager

In [1]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("user-data-dir=C:\\user-data\\kimfl")
options.add_experimental_option("detach", True)
options.add_experimental_option("excludeSwitches", ["enable-logging"])

driver = webdriver.Chrome(options=options)
driver.get("https://gcloud.csi.go.kr/cmq/main.do")



In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.webdriver.common.by import By
import re

# [단계 1] 세션 유지 (기존 driver 활용)
selenium_cookies = driver.get_cookies()
session = requests.Session()
for cookie in selenium_cookies:
    session.cookies.set(cookie['name'], cookie['value'])

detail_url = "https://gcloud.csi.go.kr/cmq/qtr/qltRqst/rqstRcvView.do"
target_rqst_nos = ["RQ-2025-242229"] 
all_details = []

for rqst_no in target_rqst_nos:
    payload = {'rqltNo': rqst_no, 'pageMode': 'recv', 'searchKey': '1', 'searchDateKey': '1'}
    try:
        response = session.post(detail_url, data=payload, timeout=100)
        response
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            row_data = {"의뢰번호": rqst_no}
            
            # [사진 54b423 분석 결과] th-td가 일렬로 나열된 모든 태그 수집
            all_tags = soup.find_all(['th', 'td'])
            
            # 수집 대상 키워드 (사진에 적힌 글자 그대로)
            targets = ["접수번호", "최종진행상태", "의뢰기관", "공사명", "발주자", "시공자"]
            
            for t in targets:
                row_data[t] = "" 
                for i, tag in enumerate(all_tags):
                    # 모든 공백과 특수문자를 제거하여 텍스트 순수 비교
                    clean_text = re.sub(r'\s+', '', tag.get_text())
                    
                    if tag.name == 'th' and t in clean_text:
                        # th를 찾으면 무조건 바로 다음(i+1) td가 정답 데이터임
                        if i + 1 < len(all_tags) and all_tags[i+1].name == 'td':
                            row_data[t] = all_tags[i+1].get_text(strip=True)
                            break

            # [사진 53ac9e 분석] 하단 처리자 추출
            row_data["특정처리자"] = ""
            hist_section = soup.find(id="rqst_hist_div")
            if hist_section:
                rows = hist_section.select("tbody tr")
                for r in rows:
                    cols = r.find_all("td")
                    if len(cols) >= 3 and "한국건설품질시험원" in cols[1].get_text():
                        row_data["특정처리자"] = cols[2].get_text(strip=True)

            all_details.append(row_data)
            print(f"[{rqst_no}] 추출 완료")
    except Exception as e:
        print(f"오류: {e}")

df = pd.DataFrame(all_details)
display(df)
# df.to_excel("final_refined_data.xlsx", index=False)




In [8]:
detail_url = "https://gcloud.csi.go.kr/cmq/qtr/qltRqst/rqstRcvView.do"
target_rqst_nos = ["RQ-2025-242229"] 
payload = {'rqltNo': rqst_no, 'pageMode': 'recv', 'searchKey': '1', 'searchDateKey': '1'}
response = session.post(detail_url, data=payload, timeout=100)
soup = BeautifulSoup(response.text, 'html.parser')
row_data = {"의뢰번호": rqst_no}
print(row_data)

{'의뢰번호': 'RQ-2025-242229'}
